In [70]:
import json
import time
from selenium import webdriver
import pandas as pd
import matplotlib.pyplot as plt
from markdownify import markdownify
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

In [71]:
def scroll_to_element_and_click(driver, web_element):
    """
    Selenium will not allow you to click an element that is not visible within the current viewpoint with the Firefox driver
    
    This function uses some javascript to scroll to a web element (so that it is in view) and then click it
    """
    x = web_element.location['x']
    y = web_element.location['y']
    scroll_by_coord = 'window.scrollTo(%s,%s);' % (
        x,
        y
    )
    scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
    driver.execute_script(scroll_by_coord)
    driver.execute_script(scroll_nav_out_of_way)
    
    actions = ActionChains(driver)
    actions.move_to_element(web_element).click().perform()  

def scrape_individual_job_detail(driver):
    job = {}

    job['url'] = driver.current_url
    job['title'] = driver.find_element_by_css_selector('div#vjs-jobtitle').text
    job['hiring_organization'] = driver.find_element_by_css_selector('div#vjs-header-jobinfo div#vjs-jobinfo div span#vjs-cn').text
    job['location'] = driver.find_element_by_css_selector('div#vjs-jobinfo div span#vjs-loc').text
    try:
        job['remuneration'] = driver.find_elements_by_css_selector('#vjs-jobinfo > div:nth-child(3) > span:nth-child(1)')[0].text
    except:
        job['remuneration'] = None

    try:
        job['job_type'] = driver.find_elements_by_css_selector('#vjs-jobinfo > div:nth-child(3) > span:nth-child(2)')[0].text
    except:
        job['remuneration'] = None

    job['job_description_html'] = driver.find_elements_by_css_selector('#vjs-desc')[0].get_attribute('innerHTML')
    job['job_description'] = driver.find_elements_by_css_selector('#vjs-desc')[0].text 
    
    return job

def scrape_indeed_job_details(driver):
    """
    scrape job details from indeed.co.uk

    args:
        driver: selenium web driver

    returns:
        array of dictionary objects containing job details
    """

    jobs = []

    while True:
        driver.get(f'https://www.indeed.co.uk/jobs?q=data+scientist')
        
        driver.implicitly_wait(0.1)

        for link in driver.find_elements_by_css_selector('.jobtitle.turnstileLink'):
            scroll_to_element_and_click(driver,link)         
            
            try:
                job = scrape_individual_job_detail(driver)
            except NoSuchElementException:
                scroll_to_element_and_click(driver,link)
                job = scrape_individual_job_detail(driver)
                
            jobs.append(job)

        next_button = driver.find_element_by_css_selector('.pagination-list > li:nth-child(6) > a:nth-child(1)')
        
        if next_button is not None:
            scroll_to_element_and_click(driver,next_button)
        else:
            break

    return pd.DataFrame(jobs)



In [72]:
driver = webdriver.Firefox()


jobs = scrape_indeed_job_details(driver)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=45345): Max retries exceeded with url: /session/db3f5a62-e827-4915-aebd-a4ca71d971a1/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f4ebfa72e80>: Failed to establish a new connection: [Errno 111] Connection refused'))